In [1]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [2]:
def similarity(s1:str, s2:str, model_name='paraphrase-mpnet-base-v2'):
    
    model = SentenceTransformer(model_name)
    
    # encode sentences to get their embeddings
    embedding1 = model.encode(s1, convert_to_tensor=True)
    embedding2 = model.encode(s2, convert_to_tensor=True)
    
    # compute similarity scores of two embeddings
    cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    
    return cosine_scores.item()

In [3]:
def topic(s1:str):
    # Returns the topic extracted from the entire output.
    
    return s1[:s1.find('.')]

In [4]:
def compare_similarity(s1:str, s2:str):
    
    print(f'String 1 : {s1}')
    print(f'String 2 : {s2}\n')
    
    print('Similarity :',similarity(s1,s2))

In [5]:
txtfile = open('../data/I2_1_1000.txt','r')
biz_ideas = [line.rstrip('\n') for line in txtfile]

In [6]:
for i in range(50):
    print(i,topic(biz_ideas[i]))

0 Home Service
1 Custom T-shirt printing
2 A beauty salon
3 Dry Cleaning
4 Con-crete Company
5 Custom Paint shop
6 Personal Care and Service
7 Bikes Rental Shop
8 Food Truck
9 Cleaning service
10 Pizza delivery and catering
11 Barbershops
12 Pet Sitting Business
13 Concrete Cutting and Grinding business
14 Auto Parts Depot
15 Bar
16 Auto shop
17 Beauty salon
18 Pet Sitting Business
19 Golf Course
20 Home Maintenance for Rent
21 Beauty Shop
22 Dental Care
23 Convenience Store
24 Bakery
25 Restaurant
26 Hair and Make-Up Salon
27 Bakery
28 Carpet and T
29 Real Estate
30 Photography Studios
31 Beauty salons
32 Beauty Salon
33 Web design Company
34 Small Retail businesses
35 Video Poker Business
36 Cleaning Supplies
37 Car Wash
38 Personal care services
39 Internet start-up
40 Jewelry Business
41 Pet Sitting
42 Restaurant
43 Construction business
44 Online business
45 Dining Out
46 Food and beverage store
47 Auto body repair
48 Video Productions
49 Cleaner/Dry cleaner


In [7]:
compare_similarity(topic(biz_ideas[13]), topic(biz_ideas[4]))

String 1 : Concrete Cutting and Grinding business
String 2 : Con-crete Company

Similarity : 0.2693554759025574


In [8]:
# Create a list of topics from the corpus

topics = []

for idea in biz_ideas:

    topics.append(topic(idea))

[print(topic_) for topic_ in topics[:10]]

Home Service
Custom T-shirt printing
A beauty salon
Dry Cleaning
Con-crete Company
Custom Paint shop
Personal Care and Service
Bikes Rental Shop
Food Truck
Cleaning service


[None, None, None, None, None, None, None, None, None, None]

In [9]:
model = SentenceTransformer('paraphrase-mpnet-base-v2')
topics_embeddings = model.encode(topics)

In [10]:
topics_embeddings.shape

(1000, 768)

In [11]:
# Top K similar ideas

topic_query = 'Beauty Parlor'
query_embedding = model.encode(topic_query)

top_k = 100

cos_scores = util.pytorch_cos_sim(query_embedding, topics_embeddings)[0]
top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

print("Sentence:", topic_query, "\n")
print(f'Top {top_k} most similar sentences in corpus:')

for idx in top_results[0:top_k]:
    print(topics[idx], "(Score: %.4f)" % (cos_scores[idx]))

Sentence: Beauty Parlor 

Top 100 most similar sentences in corpus:
Beauty salon (Score: 0.9091)
Beauty Salon (Score: 0.9091)
Beauty salon (Score: 0.9091)
Beauty Salon (Score: 0.9091)
Beauty Salon (Score: 0.9091)
Beauty Salon (Score: 0.9091)
Beauty Salon (Score: 0.9091)
Beauty Salon (Score: 0.9091)
Beauty Salon (Score: 0.9091)
Beauty Salon (Score: 0.9091)
A beauty salon (Score: 0.9033)
A Beauty Salon (Score: 0.9033)
Beauty Shop (Score: 0.8997)
Beauty Shop (Score: 0.8997)
Beauty salons (Score: 0.8859)
Hair Care Salon (Score: 0.7918)
A Hair Salon (Score: 0.7792)
A Hair Salon (Score: 0.7792)
A Hair Salon (Score: 0.7792)
Hair and Beauty salons (Score: 0.7688)
Hairdressing salon (Score: 0.7607)
Hairdressing Salon (Score: 0.7607)
Hair and Grooming Salon (Score: 0.7514)
Hair and Nail Salon (Score: 0.7510)
Hair Salons (Score: 0.7441)
Hair Salons (Score: 0.7441)
Hair Salon (Score: 0.7420)
Hair Salon (Score: 0.7420)
Hair Salon (Score: 0.7420)
Hair Salon (Score: 0.7420)
Hair Salon (Score: 0.7420)

In [12]:
# Clustering : KMeans

def kmeans(num_clusters:int, data:list):
    
    from sklearn.cluster import KMeans

    clustering_model = KMeans(n_clusters = num_clusters)
    clustering_model.fit(data)
    cluster_assignment = clustering_model.labels_

    return cluster_assignment

In [13]:
# Alternative Approach: DBSCAN (Does not seem to work well with semantics)

def dbscan(data:list):
    
    from sklearn.cluster import DBSCAN

    db_default = DBSCAN(eps = 0.0375, min_samples = 3).fit(data)
    cluster_assignment = db_default.labels_

    return cluster_assignment

In [14]:
def reduce_dims(data, alg='tsne', num_components=2):
    
    topics_red = None
    
    if alg == 'tsne':
        
        from sklearn.manifold import TSNE

        topics_red = TSNE(n_components=num_components).fit_transform(data)
    
    return topics_red

In [15]:
def print_clusters(num_clusters, cluster_assignment, topics):
    
    for i in range(num_clusters):
        print()
        print(f'Cluster {i + 1} contains:')
        clust_sent = np.where(cluster_assignment == i)
        for k in clust_sent[0]:
            print(f'- {topics[k]}')

In [16]:
# K-Means

# Define number of clusters
num_clusters = 30

# Reduce Dimentions using TSNE
topics_tsne = reduce_dims(topics_embeddings,alg='tsne',num_components=3)

# Apply K-Means Clustering
cluster_assignment = kmeans(num_clusters=num_clusters, data=topics_tsne) # data=topics_embeddings, topics_tsne

# Print clusters cohesively
print_clusters(num_clusters, cluster_assignment, topics)


Cluster 1 contains:
- Pet Sitting Business
- Pet Sitting Business
- Pet Sitting
- Pet Food and Pet Care
- Pet sitting
- Dog wash
- Pet Services
- Pet Food and Water Treatments
- Pet Shops
- Pet Shoppe
- Dog House
- Pet Shop
- A pet food store
- A Pet Shop
- Pet supplies
- Pet grooming and day care business
- Pet Clinic
- Pet Food Business
- Pet Care Business
- Dog walking
- Pet Shops
- Dog Walkers
- Pet Supplies
- Pet Shop
- Pet Sitting Business
- Dog Breeding
- Dog grooming
- Pet Shops
- Dog Day Care
- Pet Food and Pet Supplies
- Pet food manufacturing
- Dog Boarding Business
- Dog Wash
- Pet-Gone
- Pet Shops
- Dog washing
- Pet Sitting
- Pet Shops
- Dog & Cat sitting
- Pet Sitting Service Business
- Pet Clinic
- Dog grooming
- Pet Salon
- Pet Shop
- Pet Care
- Pet Sitting: This business will also need a store, and it can be a profitable business
- Pet Care Center
- Dog grooming
- Dog Food
- Dog grooming
- Pet Store
- Pet Sitting Services

Cluster 2 contains:
- Small Retail businesse

In [17]:
# DBScan

# Define number of clusters to show (note number of clusters is automatically determined)
num_clusters = 75

# Reduce Dimentions using TSNE
topics_tsne = reduce_dims(topics_embeddings,alg='tsne',num_components=3)

# Apply DBScan Clustering
cluster_assignment = dbscan(data=topics_tsne)

# Print clusters cohesively
print_clusters(num_clusters, cluster_assignment, topics)


Cluster 1 contains:
- Bar
- Bar
- Bar

Cluster 2 contains:
- Pet Sitting
- Pet sitting
- Pet Sitting

Cluster 3 contains:
- Confectionary
- Confectioneries
- Confectionary

Cluster 4 contains:
- A Hair Salon
- A Hair Salon
- A Hair Salon

Cluster 5 contains:
- Restaurant business
- Restaurant Business
- Restaurant business

Cluster 6 contains:
- A Restaurant
- A restaurant
- A Restaurant

Cluster 7 contains:
- Bicycles
- Bicycles
- Bicycles

Cluster 8 contains:
- Painting Shop
- Painting Shop
- Painting Shop

Cluster 9 contains:
- Laundry
- Laundry
- Laundry

Cluster 10 contains:
- Housecleaning
- Housecleaning
- Housecleaning

Cluster 11 contains:
- Pizza
- Pizza
- Pizza

Cluster 12 contains:
- Electrician
- Electrician
- Electrician

Cluster 13 contains:
- Bookstore
- Bookstore
- Bookstore

Cluster 14 contains:
- Personal training
- Personal Training
- Personal Training

Cluster 15 contains:
- Music store
- Music Store
- Music Store

Cluster 16 contains:
- Music Lessons
- Music Less